In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from typing import Any, Dict, Iterable, Iterator, List, Optional, Tuple, Union
import pandas as pd
import numpy as np
import tensorflow as tf

from transformers import PullData
window_size = 25
formation_window = 24
target_window = 4

GetData = PullData()

GetData.fit(ticker = 'nvda',
            start_date ='1980-01-01',
            end_date = '2022-08-04',
            interval = '1wk',
            progress = False,
            form_window = formation_window,
            target_window = target_window,
            timeperiod1 = 6,
            timeperiod2 = 12,
            timeperiod3 = 24,
            )

data_prep = GetData.transform()
 

In [2]:
from transformers import NormalizeData

NormalizeData = NormalizeData()

NormalizeData.fit(window_size=25,shuffle=False,debug=False)

data_normalized,Dates = NormalizeData.transform(data_prep)

Initial length of dataframe:  30025
Nr of formations:  1201
New length of dataframe:  30025

Done


In [3]:
from training import SplitData

SplitData = SplitData()

SplitData.fit(split_ratio=0.8, window_size=25,dates=Dates,debug=False)

x_train,x_valid,x_train_x, x_valid_x = SplitData.transform(data_normalized)
 
#print(data_normalized.shape)


x_train window 961.0
x_valid window 240.0
Done


In [4]:
from training import GetTensoredDataset

GetTensoredDataset = GetTensoredDataset()

GetTensoredDataset.fit(window_size=25,batch_size=16,train=True,debug=False)

x_train_tensors, _ = GetTensoredDataset.transform(x_train)
 
#print(data_normalized.shape)

Done


In [5]:
from training import GetTensoredDataset

GetTensoredValidDataset = GetTensoredDataset()

GetTensoredValidDataset.fit(window_size=25,batch_size=4,train=False,debug=False)

x_valid_tensors, labels = GetTensoredValidDataset.transform(x_valid)
x_valid_tensors


Done


<BatchDataset element_spec=(TensorSpec(shape=(None, 24, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [6]:
# x_valid.shape[0]/25

<b>Model Training</b>

In [7]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss',patience=6,mode='min', restore_best_weights=True)#

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=10e-15,
                              verbose=1)

model_checkpoint = ModelCheckpoint(monitor='val_loss', 
                             filepath='./nvda_80_model_checkpoint_recent.h5', 
                             save_best_only=True)

callbacks = [early_stopping,reduce_lr,model_checkpoint]

def sign_penalty(y_true,y_pred):
    penalty = 100.
    loss = tf.where(tf.less(y_true*y_pred,0),
                    penalty * tf.square(y_true-y_pred),
                    tf.square(y_true - y_pred)
                   )
    
    return(tf.reduce_mean(loss,axis=-1))

tf.keras.losses.sign_penalty = sign_penalty

Model 1

In [8]:
# tf.random.set_seed(7788)

# model = tf.keras.models.Sequential([

#     tf.keras.layers.Conv1D(filters=256, kernel_size=10,
#                       strides=1, padding="same",
#                       activation=tf.nn.selu,
#                       input_shape=[None, 7]),
# #     tf.keras.layers.Conv1D(filters=512, kernel_size=10,
# #                       strides=1, padding="same",
# #                       activation=tf.nn.selu,
# #                       input_shape=[None, 7]),
    
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     # tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
#     #tf.keras.layers.Dropout(0.2),   
#     tf.keras.layers.Dense(1024, activation=tf.nn.selu),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(1024, activation=tf.nn.selu),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(1024, activation=tf.nn.selu),
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(1024, activation=tf.nn.selu),   
#     #tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(16, activation=tf.nn.selu),
#     tf.keras.layers.Dense(16, activation=tf.nn.selu),
#     tf.keras.layers.Dense(4, activation=tf.nn.selu),
#     tf.keras.layers.Dense(3, activation=tf.nn.selu), #4
#     tf.keras.layers.Dense(1,activation=tf.nn.relu),
# ])

# #optimizer1 = tf.keras.optimizers.SGD(learning_rate=10e-7, momentum=0.9)
# #optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# #optimizer3 = tf.keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False)
# #optimizer4 = tf.keras.optimizers.Adadelta(learning_rate=0.001, rho=0.95, epsilon=1e-07, name='Adadelta')

# optimizer5 = tf.keras.optimizers.Adagrad(learning_rate=0.0001, initial_accumulator_value=0.1, epsilon=1e-07,name='Adagrad')


# model.compile(loss=sign_penalty,
#               optimizer=optimizer5,
#               )

# model.fit(train_set, epochs=120,callbacks=[callbacks],validation_data=val_dataset)


Model 2

In [9]:
tf.random.set_seed(7788)

model = tf.keras.models.Sequential([

    tf.keras.layers.Conv1D(filters=8, kernel_size=1,
                      strides=1, padding="same",
                      activation=tf.nn.selu,
                      input_shape=[None, 7]),
     tf.keras.layers.Conv1D(filters=16, kernel_size=1,
                      strides=1, padding="same",
                      activation=tf.nn.elu,
                      #input_shape=[None, 7]
                           ),
         tf.keras.layers.Conv1D(filters=32, kernel_size=10,
                      strides=1, padding="same",
                      activation=tf.nn.selu,
                      #input_shape=[None, 7]
                           ),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(3, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(3)),
    tf.keras.layers.Dense(3, activation=tf.nn.selu),
    tf.keras.layers.Dense(2, activation=tf.nn.selu),
    tf.keras.layers.Dense(1,activation=tf.nn.relu),
    ])

optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.0009, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
optimizer5 = tf.keras.optimizers.Adagrad(learning_rate=0.01, initial_accumulator_value=4, epsilon=1e-07,name='Adagrad')

model.compile(loss=sign_penalty,
              optimizer=optimizer5,
              )

model.fit(x_train_tensors, epochs=6,callbacks=[callbacks],validation_data=x_valid_tensors)



Epoch 1/6
61/61 [==============================] - 8s 54ms/step - loss: 0.0369 - val_loss: 0.0280 - lr: 0.0100
Epoch 2/6
61/61 [==============================] - 1s 18ms/step - loss: 0.0292 - val_loss: 0.0260 - lr: 0.0100
Epoch 3/6
61/61 [==============================] - 1s 20ms/step - loss: 0.0272 - val_loss: 0.0247 - lr: 0.0100
Epoch 4/6
61/61 [==============================] - 1s 23ms/step - loss: 0.0259 - val_loss: 0.0238 - lr: 0.0100
Epoch 5/6
61/61 [==============================] - 2s 25ms/step - loss: 0.0248 - val_loss: 0.0228 - lr: 0.0100
Epoch 6/6
61/61 [==============================] - 1s 23ms/step - loss: 0.0239 - val_loss: 0.0221 - lr: 0.0100


In [10]:
def model_forecast(model, series, window_size,debug):

    """
    Get model, data and window size as an input. 
    Make prediction window is subtracted by 1, since we do not need label in window, 
    label value is skipped
    """
    c = 0
    
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size-1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    
    if debug == True:
        #This block of code will print out data on which is made prediction
        for item in ds:
            c += 1
            if c <3:
                print("\n"+str(c)+ " prediction:\n ",item)
            else:
                break
            
    ds = ds.batch(1).prefetch(1)
    forecast = model.predict(ds)
    forecast2 = np.squeeze(forecast)
    return forecast2

forecast = model_forecast(model, x_valid, window_size=window_size,debug=False)
forecast[:2]

240/240 [==============================] - 2s 3ms/step


array([0.95655805, 0.91791713], dtype=float32)

In [11]:
pr = x_valid.iloc[:24,:].to_numpy()
pr = np.array([pr])
pr = np.array([pr])
pred = tf.data.Dataset.from_tensor_slices(pr)
predict = model.predict(pred)
print("Raw prediction: ",predict)

1/1 [==============================] - 0s 11ms/step
Raw prediction:  [[0.95655805]]


In [12]:
from transformers import ReverseNormalization

ReverseNormalization = ReverseNormalization()

ReverseNormalization.fit(forecasts=forecast,labels=labels,x_valid = x_valid,x_valid_x = x_valid_x,window_size=25,debug=False)

df = ReverseNormalization.transform()
df



Done
Done


,Open,High,Low,Close,EMA6,EMA12,EMA24,labels,prediction
In,,,,,,,,,
0,38.352501,40.435001,38.305000,38.457500,36.011488,33.162876,29.691369,NaN,NaN
1,38.790001,39.150002,35.875000,36.139999,36.048206,33.620895,30.207260,NaN,NaN
2,36.262501,36.875000,34.645000,36.689999,36.231575,34.093064,30.725879,NaN,NaN
3,37.435001,41.575001,37.169998,41.237499,37.661839,35.192208,31.566809,NaN,NaN
4,41.582500,42.325001,40.325001,42.025002,38.908457,36.243407,32.403464,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5995,191.389999,193.369995,168.690002,169.740005,183.115982,197.401152,214.717334,NaN,NaN
5996,160.000000,168.679993,153.279999,158.800003,176.168560,191.462513,210.243947,NaN,NaN
5997,164.750000,171.399994,158.529999,171.259995,174.766113,188.354434,207.125231,NaN,NaN
